<a href="https://githubtocolab.com/giswqs/geemap/blob/master/examples/notebooks/11_export_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

Uncomment the following line to install [geemap](https://geemap.org) if needed.

In [ ]:
#y. agbohessou
# !pip install geemap


In [12]:
import ee
import geemap
import os

In [13]:
#geemap.show_youtube('_6JOA-iiEGU')

In [14]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## Download an ee.Image

In [24]:
# %clay: https://developers.google.com/earth-engine/datasets/catalog/ISDASOIL_Africa_v1_clay_content?hl=en
image  = ee.Image('ISDASOIL/Africa/v1/ph').select([0, 1])
image_vis = {'bands': ['mean_0_20', 'mean_20_50'], 'gamma': 1.4}
Map.addLayer(image , image_vis, "ISDASOIL/Africa/v1/ph")
Map.set_plot_options(add_marker_cluster=True)

In [16]:
# Draw any shapes on the map using the Drawing tools before executing this code block
feature = Map.draw_last_feature
roi = feature.geometry()

In [27]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
filename = os.path.join(out_dir, 'clay_content.tif')

### Exporting all bands as one single image

In [14]:
image = image.clip(roi).unmask()
geemap.ee_export_image(
    image, filename=filename, scale=90, region=roi, file_per_band=False
)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\HP\Downloads\landsat.tif


### Exporting each band as one image

In [13]:
geemap.ee_export_image(
    image, filename=filename, scale=90, region=roi, file_per_band=True
)

Generating URL ...
An error occurred while downloading.
Pixel grid dimensions (44143x7176) must be less than or equal to 10000.


### Export an image to Google Drive

In [28]:
geemap.ee_export_image_to_drive(
    image, description='sahel_isda_pH_content', folder='export', region=roi, scale=100
)

Exporting sahel_isda_pH_content ...


## Download an ee.ImageCollection

In [59]:
import ee
import geemap
import os

In [60]:
# Draw any shapes on the map using the Drawing tools before executing this code block
feature = Map.draw_last_feature
roi = feature.geometry()

In [61]:
loc = roi
collection = (
    ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY')
    .filterBounds(loc)
    .filterDate('2020-01-01', '2020-01-02')
    #.filter(ee.Filter.listContains("system:band_names", "soil_temperature_level_1"))
)

In [62]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')

In [63]:
print(collection.aggregate_array('system:index').getInfo())

['20200101T00', '20200101T01', '20200101T02', '20200101T03', '20200101T04', '20200101T05', '20200101T06', '20200101T07', '20200101T08', '20200101T09', '20200101T10', '20200101T11', '20200101T12', '20200101T13', '20200101T14', '20200101T15', '20200101T16', '20200101T17', '20200101T18', '20200101T19', '20200101T20', '20200101T21', '20200101T22', '20200101T23']


In [ ]:
geemap.ee_export_image_collection(collection, out_dir=out_dir,region=roi,scale=10000)

In [ ]:
# for large data
# geemap.ee_export_image_collection_to_drive(collection, folder='export', region=roi, scale=100)

## Extract pixels as a Numpy array

In [1]:
import ee
import geemap
import numpy as np
import matplotlib.pyplot as plt

img = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_038029_20180810').select(['B4', 'B5', 'B6'])

aoi = ee.Geometry.Polygon(
    [[[-110.8, 44.7], [-110.8, 44.6], [-110.6, 44.6], [-110.6, 44.7]]], None, False
)

rgb_img = geemap.ee_to_numpy(img, region=aoi)
print(rgb_img.shape)

EEException: Earth Engine client library not initialized. Run `ee.Initialize()`

In [ ]:
# Scale the data to [0, 255] to show as an RGB image.
# Adapted from https://bit.ly/2XlmQY8. Credits to Justin Braaten
rgb_img_test = (255 * ((rgb_img[:, :, 0:3] - 100) / 3500)).astype('uint8')
plt.imshow(rgb_img_test)
plt.show()